<a href="https://colab.research.google.com/github/Herais/NLP_Learning_by_Selective_Data/blob/main/Open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project directory
import os
if not os.path.exists('/content/drive/MyDrive/Github'):
  !mkdir Github
path_github   = '/content/drive/MyDrive/Github'
os.chdir(path_github)

# git clone
#!git clone https://github.com/Herais/NLP_Learning_by_Selective_Data.git
path_wd = path_github + '/' + 'NLP_Learning_by_Selective_Data'
os.chdir(path_wd)
!pwd

# USE GPU FOR TRAINING AND PREDICTION

Mounted at /content/drive
/content/drive/MyDrive/Github/NLP_Learning_by_Selective_Data


In [ ]:
# INSTALL REQUIRED PACKAGES
!pip install -r requirements.txt
%tensorflow_version 1.x

# Download pretrained BERT model if necesary
path_pretrained_model = '/pretrained_model'
path_bert_chinese = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12'
if not os.path.exists(path_bert_chinese):
 # Download Model
  !wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip -P './pretrained_model/'
  %cd pretrained_model
  !unzip chinese_L-12_H-768_A-12.zip
  !rm -f chinese_L-12_H-768_A-12.zip
  os.chdir(path_wd)
  !pwd

# Download Trained Weights
# 1. model StarChaser
# 2. model Scholar 
# 3. model Knowitall
path_models = '/models'
if not os.path.exists(path_wd + path_models + 'best_model_StarChaser.weights'):
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1v8T0-SWwbNYvoV1fMPfq76e-mQPIHpzt' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1v8T0-SWwbNYvoV1fMPfq76e-mQPIHpzt"  && rm -rf /tmp/cookies.txt -P './models/'
if not os.path.exists(path_wd + path_models + 'best_model_Scholar.weights'):
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10rvkPWHcdQwY4RKeNdpAJ8k7TCuALEn9' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10rvkPWHcdQwY4RKeNdpAJ8k7TCuALEn9"  && rm -rf /tmp/cookies.txt -P './models/'
if not os.path.exists(path_wd + path_models + 'best_model_Knowitall.weights'):
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1QyDDg3oIXkmQGEhl4wmRZ5M_-1T5hG_t' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1QyDDg3oIXkmQGEhl4wmRZ5M_-1T5hG_t"  && rm -rf /tmp/cookies.txt -P './models/'

In [ ]:
# LOAD AND EXPLORE DATASETS

# Import Libaries
import numpy as np
import pandas as pd

# Path
path_datasets ='/datasets'

# Load Dataset: Movie Title and Summary
path_dataset_movie = '/Movie_Title_and_Summary'
df_movie = pd.read_json(path_wd + path_datasets + path_dataset_movie + '/train.json', orient='records', lines=True)
df_movie.head(10)

# Load Dataset: Computer Research Literature (CSL) Title and Abstract
path_dataset_csl = '/CSL'
df_csl = pd.read_json(path_wd + path_datasets + path_dataset_csl + '/train.json', orient='records', lines=True)
df_csl.head(10)

In [6]:
# SET MODEL PARAMETERS

# Set Model Parameter
epochs = 10
batch_size = 8
maxlen = 256
topk = 1



# Set Pretrained Model Parameters (BERT)
config_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/vocab.txt'


# Load Vocab
from bert4keras.tokenizers import Tokenizer, load_vocab

token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)

In [4]:
# LOAD TRAINED MODELS
from tensorflow import keras
from bert4keras.models import build_transformer_model

path_models = '/models'

# the Star Chaser Model trained with movie dataset only
model_StarChaser = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # include only tokens in keep tokens
)
model_StarChaser.load_weights(path_wd + path_models + '/best_model_StarChaser.weights')

# the Scholar Model trained with Computer Science Literature only
model_Scholar = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
)
model_Scholar.load_weights(path_wd + path_models + '/best_model_Scholar.weights')

# the Know-it-all Model Trained with both movie and cscl dataset
model_Knowitall = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
) 
model_Knowitall.load_weights(path_wd + path_models + '/best_model_Knowitall.weights')


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


"\n# the Scholar Model trained with Computer Science Literature only\nmodel_Scholar = build_transformer_model(\n    config_path,\n    checkpoint_path,\n    application='unilm',\n    keep_tokens=keep_tokens,   # include only tokens in keep tokens\n)\nmodel_Scholar.load_weights(path_wd + path_models + '/best_model_Scholar.weights')\n\n# the Know-it-all Model Trained with both movie and cscl dataset\nmodel_Knowitall = build_transformer_model(\n    config_path,\n    checkpoint_path,\n    application='unilm',\n    keep_tokens=keep_tokens,   # include only tokens in keep tokens\n) \nmodel_Knowitall.load_weights(path_wd + path_models + '/best_model_Knowitall.weights')\n"

In [11]:
# PREDICTION

# Load Library
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder

# Set Path
path_datasets ='/datasets'
path_predictions = '/predictions'
path_test = path_wd + path_datasets + path_dataset_csl + '/test.json'
path_prediction = path_wd + path_predictions + '/Model_StarChaser_predict_CSLTitle.json'

# Load Test Data
test_data = []
with open(path_test, encoding='utf-8') as f:
    for l in f:
        test_data.append(json.loads(l.strip()))

# Decoder: seq2seq
class AutoTitle(AutoRegressiveDecoder):
    #seq2seq decoder
    @AutoRegressiveDecoder.wraps(default_rtype='probas')
    def predict(self, inputs, output_ids, states):
        token_ids, segment_ids = inputs
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.concatenate([segment_ids, np.ones_like(output_ids)], 1)
        # Change model name here:
        return model_StarChaser.predict([token_ids, segment_ids])[:, -1]

    def generate(self, text, topk=1):
        max_c_len = maxlen - self.maxlen
        token_ids, segment_ids = tokenizer.encode(text, maxlen=max_c_len)
        output_ids = self.beam_search([token_ids, segment_ids],
                                      topk=topk)  # beam search
        return tokenizer.decode(output_ids)

autotitle = AutoTitle(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)

# Write to Prediction Path
with open(path_prediction ,'w') as f:
    for data in test_data[:100]:
        pred_title = ''.join(autotitle.generate(data["abst"], topk)).lower()
        output = {}
        output["id"] = data["id"]
        output["title"] = pred_title
        f.write(json.dumps(output, ensure_ascii = False))
        f.write('\n')

In [ ]:
# RESULT ANALYSIS

In [ ]:
# TRAIN MORE MODELS
# Skip to LOAD TRAINED MODELS to use existing models for prediction

# Load Libraries
from __future__ import print_function
import numpy as np
import json
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
#from tensorflow.keras.models import Model
from keras.models import Model
from rouge import Rouge  # pip install rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import model as md
import utils

# Load train data
path_train = path_wd + path_datasets + '/CSL/train.json'
path_dev = path_wd + path_datasets + '/CSL/dev.json'
train_data = md.load_data(path_train)
valid_data = md.load_data(path_dev)
# Could use pandas, but pandas has limitation on cell string length

# Initialize Model with Pretrained BERT
token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)
model = md.build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # 只保留keep_tokens中的字，精简原字
)

autotitle = AutoTitle(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)
evaluator = Evaluator()
output = CrossEntropy(2)(model.inputs + model.outputs)
model = Model(model.inputs, output)
train_generator = md.data_generator(train_data, batch_size)
model.compile(optimizer=md.Adam(1e-5))
#model.summary()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)

# Save Trained Model
path_models = '/models'
utils.save_model_with_dt(path_wd + path_models, model, 'new_train_test')